In [6]:
import sys
from pathlib import Path
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV, LeaveOneGroupOut
import time

# Set ROOT path to access other directories in project
ROOT = Path.cwd().parent
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

import SnowDepth.data_loader as DL
import SnowDepth.data_splitter as DS
import SnowDepth.optimal_features as OF
from SnowDepth.config import HOLDOUT_AOI
from SnowDepth.config import SEED
from SnowDepth.config import FEATURE_NAMES

In [7]:
# Path to TIFF files
data_dir = ROOT/"data"/"tif_files"

# Select holdout AOI
holdout_aoi=HOLDOUT_AOI

# Select max amount of features to select from FF algos
top_k = 10

# Load dataframe
df = DL.build_df(str(data_dir), drop_invalid=True, upper_threshold=3)

# Development dataframe we will use for training models
dev_df  = df[df['aoi_name'] != holdout_aoi].copy()

In [8]:
# Run Feature filtering algorithms
ff_algos = OF.optimal_feature_sets(dev_df, top_k=10, n_per_aoi=20000)

base_cols = ["aoi_name", "row", "col", "SD"]

# HSIC
dev_df_HSIC  = dev_df[base_cols + ff_algos["HSIC"]].copy()
# PCC
dev_df_PCC  = dev_df[base_cols + ff_algos["PCC"]].copy()
# MI
dev_df_MI  = dev_df[base_cols + ff_algos["MI"]].copy()
# ALL features (for Ablation study)
dev_df_ALL  = dev_df[base_cols + FEATURE_NAMES].copy()

Block HSIC Lasso B = 20.
M set to 3.
Using Gaussian kernel for the features, Gaussian kernel for the outcomes.
HSIC selected: ['Veg_height', 'IAFE', 'Gamma_VH_RTC', 'cos_Aspect', 'Gamma_VV_RTC', 'Sigma_ratio', 'Beta_VH', 'Gamma_RTC_ratio', 'Elevation', 'Slope']
PCC selected: ['Veg_height', 'IAFE', 'cos_Aspect', 'Gamma_VH_RTC', 'Gamma_RTC_ratio', 'sin_Aspect', 'Beta_VH']
MI selected): ['IAFE', 'Elevation', 'Veg_height', 'Gamma_VH_RTC', 'Gamma_RTC_sum', 'Gamma_VV_RTC', 'Beta_VH', 'cos_Aspect']


Split data for training XGBoost

In [9]:
# HSIC
X_dev_HSIC, y_dev_HSIC, groups_HSIC = DS.ML_split(
    dev_df=dev_df_HSIC,
    pxs_per_aoi=10000
)
# PCC
X_dev_PCC, y_dev_PCC, groups_PCC = DS.ML_split(
    dev_df=dev_df_PCC,
    pxs_per_aoi=10000
)
# MI
X_dev_MI, y_dev_MI, groups_MI = DS.ML_split(
    dev_df=dev_df_MI,
    pxs_per_aoi=10000
)
# ALL
X_dev_ALL, y_dev_ALL, groups_ALL = DS.ML_split(
    dev_df=dev_df_ALL,
    pxs_per_aoi=10000
)

Total samples: 50000 across 5 AOIs
Features used: ['Veg_height', 'IAFE', 'Gamma_VH_RTC', 'cos_Aspect', 'Gamma_VV_RTC', 'Sigma_ratio', 'Beta_VH', 'Gamma_RTC_ratio', 'Elevation', 'Slope']
X_dev shape: (50000, 10)
Total samples: 50000 across 5 AOIs
Features used: ['Veg_height', 'IAFE', 'cos_Aspect', 'Gamma_VH_RTC', 'Gamma_RTC_ratio', 'sin_Aspect', 'Beta_VH']
X_dev shape: (50000, 7)
Total samples: 50000 across 5 AOIs
Features used: ['IAFE', 'Elevation', 'Veg_height', 'Gamma_VH_RTC', 'Gamma_RTC_sum', 'Gamma_VV_RTC', 'Beta_VH', 'cos_Aspect']
X_dev shape: (50000, 8)
Total samples: 50000 across 5 AOIs
Features used: ['Sigma_VH', 'Sigma_VV', 'Gamma_VH', 'Gamma_VV', 'Beta_VH', 'Beta_VV', 'Gamma_VH_RTC', 'Gamma_VV_RTC', 'Sigma_sum', 'Gamma_sum', 'Beta_sum', 'Gamma_RTC_sum', 'Sigma_diff', 'Gamma_diff', 'Beta_diff', 'Gamma_RTC_diff', 'Sigma_ratio', 'Gamma_ratio', 'Beta_ratio', 'Gamma_RTC_ratio', 'LIA', 'IAFE', 'Elevation', 'Slope', 'sin_Aspect', 'cos_Aspect', 'Veg_height']
X_dev shape: (50000, 27)


Train RF and tune hyperparameters

In [10]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV, LeaveOneGroupOut, cross_validate
import numpy as np, time

param_dist = {
    "n_estimators": [200, 300, 400, 600],
    "max_depth": [12, 14, 16, 18],
    "max_features": ["sqrt", 0.3, 0.5, 3],
    "min_samples_leaf": [2, 5, 10, 20, 50, 0.005, 0.01],
    "max_samples": [0.3, 0.5],
}

def run_rf_search(tag, X, y, groups, n_iter=30, seed=SEED):
    rf = RandomForestRegressor(
        random_state=seed,
        bootstrap=True,
        n_jobs=-1
    )
    logo = LeaveOneGroupOut()
    search = RandomizedSearchCV(
        estimator=rf,
        param_distributions=param_dist,
        n_iter=n_iter,
        cv=logo,
        scoring="neg_root_mean_squared_error",  # select by RMSE
        n_jobs=-1,
        verbose=2,
        random_state=seed,
        refit=True
    )

    t0 = time.time()
    search.fit(X, y, groups=groups)
    elapsed = (time.time() - t0) / 60.0

    best_rmse = -search.best_score_
    best_params = search.best_params_

    # With best hyperparams, compute both RMSE and MAE via cross-validate
    best_rf = RandomForestRegressor(**best_params, random_state=seed, n_jobs=-1, bootstrap=True)
    scores = cross_validate(
        best_rf, X, y, groups=groups, cv=logo, n_jobs=-1, verbose=0,
        scoring={
            "rmse": "neg_root_mean_squared_error",
            "mae":  "neg_mean_absolute_error",
        }
    )
    cv_rmse = -np.mean(scores["test_rmse"])
    cv_mae  = -np.mean(scores["test_mae"])

    print(f"\nResults - RF with {tag} feature set")
    print(f"Best hyperparameters: {best_params}")
    print(f"Best CV RMSE (search score): {best_rmse:.4f}")
    print(f"Cross-validated (re-fit) RMSE: {cv_rmse:.4f} | MAE: {cv_mae:.4f}")
    print(f"Training time: {elapsed:.2f} minutes")

    return tag, cv_rmse, cv_mae, best_params, elapsed


In [11]:
rf_results = {}  # tag -> (rmse, mae, params)
timings = {}

for tag, X, y, g in [
    ("HSIC", X_dev_HSIC, y_dev_HSIC, groups_HSIC),
    ("PCC",  X_dev_PCC,  y_dev_PCC,  groups_PCC),
    ("MI",   X_dev_MI,   y_dev_MI,   groups_MI),
    ("ALL",  X_dev_ALL,  y_dev_ALL,  groups_ALL),  # include ALL-features set
]:
    tag, rmse, mae, params, tmin = run_rf_search(tag, X, y, g)
    rf_results[tag] = (rmse, mae, params)
    timings[tag] = tmin


Fitting 5 folds for each of 30 candidates, totalling 150 fits

Results - RF with HSIC feature set
Best hyperparameters: {'n_estimators': 300, 'min_samples_leaf': 10, 'max_samples': 0.5, 'max_features': 0.5, 'max_depth': 18}
Best CV RMSE (search score): 0.4871
Cross-validated (re-fit) RMSE: 0.4871 | MAE: 0.4107
Training time: 4.85 minutes
Fitting 5 folds for each of 30 candidates, totalling 150 fits

Results - RF with PCC feature set
Best hyperparameters: {'n_estimators': 300, 'min_samples_leaf': 20, 'max_samples': 0.5, 'max_features': 0.3, 'max_depth': 14}
Best CV RMSE (search score): 0.4906
Cross-validated (re-fit) RMSE: 0.4906 | MAE: 0.4081
Training time: 3.64 minutes
Fitting 5 folds for each of 30 candidates, totalling 150 fits

Results - RF with MI feature set
Best hyperparameters: {'n_estimators': 200, 'min_samples_leaf': 5, 'max_samples': 0.5, 'max_features': 3, 'max_depth': 16}
Best CV RMSE (search score): 0.4930
Cross-validated (re-fit) RMSE: 0.4930 | MAE: 0.4158
Training time:

In [12]:
print("\nCross-validation results (Random Forest):")
for name, (rmse, mae, params) in rf_results.items():
    print(f"\n{name} — CV RMSE: {rmse:.4f} | CV MAE: {mae:.4f}")
    print(f"{name} — Best hyperparameters: {params}")
    print(f"{name} — Training time: {timings[name]:.2f} minutes")

best_rf_method = min(rf_results, key=lambda k: rf_results[k][0])
best_rf_rmse, best_rf_mae, best_rf_params = rf_results[best_rf_method]

print(f"\nBest feature set with RF: {best_rf_method} "
      f"(CV RMSE = {best_rf_rmse:.4f}, CV MAE = {best_rf_mae:.4f}, "
      f"time = {timings[best_rf_method]:.2f} min)")
print(f"Best RF hyperparameters: {best_rf_params}")



Cross-validation results (Random Forest):

HSIC — CV RMSE: 0.4871 | CV MAE: 0.4107
HSIC — Best hyperparameters: {'n_estimators': 300, 'min_samples_leaf': 10, 'max_samples': 0.5, 'max_features': 0.5, 'max_depth': 18}
HSIC — Training time: 4.85 minutes

PCC — CV RMSE: 0.4906 | CV MAE: 0.4081
PCC — Best hyperparameters: {'n_estimators': 300, 'min_samples_leaf': 20, 'max_samples': 0.5, 'max_features': 0.3, 'max_depth': 14}
PCC — Training time: 3.64 minutes

MI — CV RMSE: 0.4930 | CV MAE: 0.4158
MI — Best hyperparameters: {'n_estimators': 200, 'min_samples_leaf': 5, 'max_samples': 0.5, 'max_features': 3, 'max_depth': 16}
MI — Training time: 3.73 minutes

ALL — CV RMSE: 0.4883 | CV MAE: 0.4123
ALL — Best hyperparameters: {'n_estimators': 300, 'min_samples_leaf': 10, 'max_samples': 0.5, 'max_features': 0.5, 'max_depth': 18}
ALL — Training time: 9.36 minutes

Best feature set with RF: HSIC (CV RMSE = 0.4871, CV MAE = 0.4107, time = 4.85 min)
Best RF hyperparameters: {'n_estimators': 300, 'min